### Libraries

In [16]:
import pandas as pd
import numpy as np 

import warnings
import os

from unidecode import unidecode

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
warnings.filterwarnings("ignore")
os.chdir("C:\\Users\\joaos\\Documents\\GitHub\\pred_se_ml\data")

### Shadow economy data from Medina, L., & Schneider, F. (2017)

In [4]:
# Importing shadow economy data from Medina, L., & Schneider, F. (2017)
se_medina = pd.read_excel("shadow_economy_medina.xlsx")
se_medina.head()

,Country,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Albania,43.18,40.18,39.45,40.07,39.18,37.07,37.59,38.16,36.04,35.3,36.04,33.67,32.64,31.72,30.89,29.58,28.53,27.12,26.91,26.10,25.41,25.52,25.68,25.78,26.21
1,Algeria,34.96,36.14,38.16,38.88,37.64,37.68,37.18,38.28,36.11,34.2,33.58,31.90,29.62,27.76,24.93,24.44,24.21,24.07,25.90,25.89,27.37,26.94,25.98,25.74,23.98
2,Angola,50.17,47.80,55.43,50.48,52.47,46.30,50.48,49.21,48.64,48.8,46.14,48.40,48.86,46.81,43.84,41.23,37.13,35.26,36.25,36.54,36.49,36.60,35.92,34.53,35.25
3,Argentina,25.22,24.41,26.59,26.22,27.18,25.32,25.20,24.00,25.83,25.4,26.94,26.19,25.37,24.32,23.21,22.63,21.93,21.87,22.97,21.64,20.80,21.62,21.57,22.02,24.99
4,Armenia,46.65,49.50,48.63,44.66,47.14,47.48,46.41,45.81,46.85,46.6,47.61,44.11,42.08,43.57,41.03,41.38,39.47,35.39,41.04,40.14,38.46,35.52,34.56,34.78,35.96


In [6]:
# Columns' names
se_medina.columns

Index(['Country ',       1991,       1992,       1993,       1994,       1995,
             1996,       1997,       1998,       1999,       2000,       2001,
             2002,       2003,       2004,       2005,       2006,       2007,
             2008,       2009,       2010,       2011,       2012,       2013,
             2014,       2015],
      dtype='object')

In [8]:
# Removing the blank space and renaming the Country column
se_medina = se_medina.rename(columns={'Country ': 'country'})

# Reshaping from wide format to long format
se_medina = se_medina.set_index(se_medina['country']).stack().reset_index(name='se_medina').rename(columns={'level_2': 'year'})
se_medina.head()

,country,level_1,se_medina
0,Albania,country,Albania
1,Albania,1991,43.18
2,Albania,1992,40.18
3,Albania,1993,39.45
4,Albania,1994,40.07


In [9]:
# Renaming the column 'level_1' to 'year'
se_medina = se_medina.rename(columns={'level_1':'year'})

# Converting the 'se_medina' column to numeric
se_medina["se_medina"] = pd.to_numeric(se_medina.se_medina, errors = 'coerce')

# Droping 'na' observations
se_medina = se_medina.dropna()

Due to different data sources, the countries names may vary from one source to another, bellow some countries' names going to be renamed for future data joining

In [12]:
se_medina['country'] = [x.replace('CentralAfricanRepublic', 'central_african_republic') for x in se_medina['country']]
se_medina['country'] = [x.replace('Congo, Dem, Rep,', 'congo_dem_rep') for x in se_medina['country']]
se_medina['country'] = [x.replace('GuineaBissau', 'guinea_bissau') for x in se_medina['country']]
se_medina['country'] = [x.replace('Hong Kong SAR, China', 'hong_kong') for x in se_medina['country']]
se_medina['country'] = [x.replace('Syrian Arab, Rep,', 'Syria') for x in se_medina['country']]
se_medina['country'] = [x.replace('Brunei Darussalam', 'brunei') for x in se_medina['country']]
se_medina['country'] = [x.replace('swaziland', 'eswatini') for x in se_medina['country']]

In [17]:
# Selecting only the first name of each country
se_medina['country'] = se_medina['country'].str.split(',').str[0].str.strip()

# Only lowercase letters
se_medina['country'] = se_medina['country'].str.lower()

# Removing accentuation
se_medina['country'] = se_medina['country'].apply(lambda x: unidecode(x))

# replacing whitespace with _ in country
se_medina['country'] = [x.replace(' ', '_') for x in se_medina['country']] 